In [1]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logvar = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv_mu(x, edge_index), self.conv_logvar(x, edge_index)


class VGAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VGAE, self).__init__()
        self.encoder = Encoder(in_channels, out_channels)

    def reparameterize(self, mu, logvar):
        if self.training:
            return mu + torch.randn_like(logvar) * torch.exp(logvar)
        else:
            return mu

    def forward(self, x, edge_index):
        mu, logvar = self.encoder(x, edge_index)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar


def loss_function(pos_pred, neg_pred, mu, logvar):
    recon_loss = -torch.mean(torch.log(pos_pred + 1e-15) + torch.log(1 - neg_pred + 1e-15))
    KLD_loss = -0.5 / mu.size(0) * torch.mean(
        torch.sum(1 + 2 * logvar - mu.pow(2) - logvar.exp().pow(2), dim=1)
    )
    return recon_loss + KLD_loss


/Users/pritishmishra/opt/anaconda3/envs/torch_gpu_arm64/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'torch_geometric'

In [7]:
import dgl.nn as dglnn
import torch.nn.functional as F
from torch import nn

class Encoder(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(Encoder, self).__init__()
        self.conv1 = dglnn.GraphConv(in_feats, 2 * out_feats, activation=F.relu)
        self.conv2 = dglnn.GraphConv(2 * out_feats, 2 * out_feats)  # output 2 * out_feats

    def forward(self, g, features):
        h = self.conv1(g, features)
        h = self.conv2(g, h)
        mu, logvar = torch.chunk(h, 2, dim=1)  # Split the output into mu and logvar
        return mu, logvar

class VGAE(nn.Module):
    def __init__(self, g, in_feats, latent_feats):
        super(VGAE, self).__init__()
        self.g = g
        self.encoder = Encoder(in_feats, latent_feats)
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(logvar).pow(0.5)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, features):
        mu, logvar = self.encoder(self.g, features)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar


In [3]:
def train(model, graph, features, epochs=200):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        model.train()  # ensure the model is in training mode
        z, mu, logvar = model(features)
        
        # Compute the reconstruction loss and the KL divergence
        recons_loss = F.binary_cross_entropy_with_logits(model.decode(z)[graph.edge_index[0], graph.edge_index[1]], torch.ones(graph.num_edges()).to(device))
        kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

        loss = recons_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch:', epoch+1, ', Loss:', loss.item())


In [4]:
def generate_embeddings(model, graph, features):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No need to track gradients
        z, _, _ = model(graph, features)
    return z

In [5]:
import dgl
import torch
from dgl.data import CoraGraphDataset

# Load the Cora dataset
data = CoraGraphDataset()
g = data[0]

# Prepare the input features and labels
features = g.ndata['feat']
labels = g.ndata['label']

# Normalize the features (this is often a good preprocessing step)
features = features / features.norm(dim=1, keepdim=True)

# Initialize the model with the appropriate dimensions
# Cora has 1433 input features, and let's use 50 for the latent space dimension
model = VGAE(g, 1433, 50)

# Move everything to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
g = g.to(device)
features = features.to(device)
labels = labels.to(device)

# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 200

for epoch in range(epochs):
    model.train()  # ensure the model is in training mode
    z, mu, logvar = model(features)

    # Compute adjacency matrix
    adj = g.adjacency_matrix().to_dense()

    # Compute the reconstruction loss and the KL divergence
    recons_loss = F.binary_cross_entropy_with_logits(model.decode(z), adj)
    kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    loss = recons_loss + kl_div
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch:', epoch+1, ', Loss:', loss.item())

# Generate node embeddings
embeddings = model(features)[0].detach().cpu().numpy()  # get the latent representations and move to cpu


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


/Users/pritishmishra/opt/anaconda3/envs/torch_gpu_arm64/lib/python3.9/site-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)


FileNotFoundError: Cannot find DGL C++ sparse library at /Users/pritishmishra/opt/anaconda3/envs/torch_gpu_arm64/lib/python3.9/site-packages/dgl/dgl_sparse/libdgl_sparse_pytorch_1.14.0.dev20221204.dylib

In [8]:
torch.__version__

'1.14.0.dev20221204'

In [13]:
# Module imports
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import EXPLAINABLEENSEMBLELINKPREDICTION.eelp.utils.args # Import arguments from args.py

2.0.1


ModuleNotFoundError: No module named 'EXPLAINABLEENSEMBLELINKPREDICTION'

In [11]:
# Defining global helper functions
def dot_product_decode(Z):
    A_pred = torch.sigmoid(torch.matmul(Z, Z.t()))
    return A_pred

def glorot_init(input_dim, output_dim):
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = torch.rand(input_dim, output_dim) * 2 * init_range - init_range
    return nn.Parameter(initial)

# Defining the model
class VGAE(nn.Module):
    # Defining the constructor
    def __init__(self, adj):
        super(VGAE, self).__init__()
        self.base_gcn = GraphConvSparse(args.input_dim, args.hidden1_dim, adj)
        self.gcn_mean = GraphConvSparse(args.hidden1_dim, args.hidden2_dim, adj, activation=lambda x: x)
        self.gcn_logstddev = GraphConvSparse(args.hidden1_dim, args.hidden2_dim, adj, activation=lambda x: x)

    # Defining the encoder
    def encode(self, X):
        hidden = self.base_gcn(X)
        self.mean = self.gcn_mean(hidden)
        self.logstd = self.gcn_logstddev(hidden)
        gaussian_noise = torch.randn(X.size(0), args.hidden2_dim)  # Gaussian noise
        sampled_z = gaussian_noise * torch.exp(self.logstd) + self.mean  # Reparameterization trick
        return sampled_z
    
    # Defining the forward layer
    def forward(self, X):
        z = self.encode(X)
        return z
    
class GraphConvSparse(nn.Module):
    def __init__(self, input_dim, output_dim, adj, activation=F.relu, **kwargs):
        super(GraphConvSparse, self).__init__(**kwargs)
        self.weight = glorot_init(input_dim, output_dim)  # Weight initialization
        self.adj = adj  # Adjacency matrix
        self.activation = activation  # Activation function
    
    def forward(self, inputs):
        x = inputs
        x = torch.mm(x, self.weight)  # Matrix multiplication
        x = torch.mm(self.adj, x)  # Convolution operation
        outputs = self.activation(x)  # Activation
        return outputs
    
class GAE(nn.Module):
    def __init__(self, adj):
        super(GAE, self).__init__()
        self.base_gcn = GraphConvSparse(args.input_dim, args.hidden1_dim, adj)
        self.gcn_mean = GraphConvSparse(args.hidden1_dim, args.hidden2_dim, adj, activation=lambda x: x)

    def encode(self, X):
        hidden = self.base_gcn(X)
        z = self.mean = self.gcn.mean(hidden)
        return z
    
    def forward(self, X):
        z = self.encode(X)
        A_pred = dot_product_decode(z)
        return A_pred

ModuleNotFoundError: No module named 'args'